# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [39]:
import logging
import azureml
import joblib
import json
import os

import pandas as pd
import numpy as np

from azureml.core import Environment
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import DockerConfiguration
from azureml.core.dataset import Dataset


from azureml.core import Workspace, Experiment

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.core.runconfig import EnvironmentDefinition, CondaDependencies

from azureml.train.hyperdrive import (
    BayesianParameterSampling,
    HyperDriveConfig,
    quniform,
    uniform,
    choice,
    PrimaryMetricGoal,
)

from azureml.widgets import RunDetails

from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model

from hyperdrive.train import create_features

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.42.0


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'hyperdrive-lgbm'

experiment=Experiment(ws, experiment_name)
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-200878
Azure region: southcentralus
Subscription id: 5a4ab2ba-6c51-4805-8155-58759ad589d8
Resource group: aml-quickstarts-200878


In [3]:
# Create or attach a compute cluster

cluster_name = "canina-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('This compute target already exists.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",
                                                          max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    print('A new compute target has been created.')
    
compute_target.wait_for_completion(show_output=True, min_node_count=0, timeout_in_minutes=10)

A new compute target has been created.
InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
train_dataset = Dataset.get_by_name(ws, name='train')
train_id = train_dataset.id

In [5]:
auxi_dataset = Dataset.get_by_name(ws, name='auxi')
auxi_id = auxi_dataset.id

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [6]:
# Configure Docker environment 
env = EnvironmentDefinition()
env.python.user_managed_dependencies = False
env.python.conda_dependencies = CondaDependencies.create(
    conda_packages=["pandas", "numpy", "scipy", "scikit-learn", "lightgbm", "joblib"],
    python_version="3.6.2"
)
env.python.conda_dependencies.add_channel("conda-forge")

docker_config = DockerConfiguration(use_docker=True)

In [7]:
# Training script
script_folder = "./hyperdrive"
train_script_name = "train.py"

src = ScriptRunConfig(source_directory=script_folder,
                      script=train_script_name,
                      arguments=["--train", train_id, "--auxi", auxi_id],
                      compute_target=compute_target,
                      environment=env,
                      docker_runtime_config=docker_config)

# Create the different params used during training
param_sampling = BayesianParameterSampling(
    {
        "--num-leaves": quniform(8, 128, 1),
        "--min-data-in-leaf": quniform(20, 500, 10),
        "--learning-rate": choice(
            1e-4, 1e-3, 5e-3, 1e-2, 1.5e-2, 2e-2, 3e-2, 5e-2, 1e-1
        ),
        "--feature-fraction": uniform(0.2, 1),
        "--bagging-fraction": uniform(0.1, 1),
        "--bagging-freq": quniform(1, 20, 1),
        "--max-rounds": quniform(50, 2000, 10),
        "--max-lag": quniform(3, 40, 1),
        "--window-size": quniform(3, 40, 1)
    }
)

hyperdrive_run_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=param_sampling,
    primary_metric_name="MAPE",
    primary_metric_goal=PrimaryMetricGoal.MINIMIZE,
    max_total_runs=25,
    max_concurrent_runs=4
)

For best results with Bayesian Sampling we recommend using a maximum number of runs greater than or equal to 20 times the number of hyperparameters being tuned. Recommendend value:180.


In [8]:
# Submit the experiment
hyperdrive_run = experiment.submit(config=hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [10]:
# We need to wait before retrieving the best model
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_55e6ebb7-526e-4ee4-9441-ac4044c9d829
Web View: https://ml.azure.com/runs/HD_55e6ebb7-526e-4ee4-9441-ac4044c9d829?wsid=/subscriptions/5a4ab2ba-6c51-4805-8155-58759ad589d8/resourcegroups/aml-quickstarts-200878/workspaces/quick-starts-ws-200878&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Execution Summary
RunId: HD_55e6ebb7-526e-4ee4-9441-ac4044c9d829
Web View: https://ml.azure.com/runs/HD_55e6ebb7-526e-4ee4-9441-ac4044c9d829?wsid=/subscriptions/5a4ab2ba-6c51-4805-8155-58759ad589d8/resourcegroups/aml-quickstarts-200878/workspaces/quick-starts-ws-200878&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_55e6ebb7-526e-4ee4-9441-ac4044c9d829',
 'target': 'canina-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-07-12T18:17:32.498868Z',
 'endTimeUtc': '2022-07-12T19:04:23.028862Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "MAPE", "goal": "minimize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '161ff105-d2a5-4fee-b6f9-1d4f84e031e6',
  'user_agent': 'python/3.8.5 (Linux-5.4.0-1083-azure-x86_64-with-glibc2.10) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.42.0',
  'space_size': 'infinite_space_size',
  'score': '30.704403788529106',
  'best_child_run_id': 'HD_55e6ebb7-526e-4ee4-9441-ac4044c9d829_2',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_55e6ebb7-526e-4ee4-9441-ac4044c9d829_2'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg200878.blob.core.wi

In [11]:
# Provide details about the different runs and their performance
hyperdrive_run.get_metrics()

{'HD_55e6ebb7-526e-4ee4-9441-ac4044c9d829_24': {'MAPE': 41.64674231959689},
 'HD_55e6ebb7-526e-4ee4-9441-ac4044c9d829_21': {'MAPE': 36.34141010456536},
 'HD_55e6ebb7-526e-4ee4-9441-ac4044c9d829_22': {'MAPE': 41.97461946037907},
 'HD_55e6ebb7-526e-4ee4-9441-ac4044c9d829_23': {'MAPE': 35.702473119493625},
 'HD_55e6ebb7-526e-4ee4-9441-ac4044c9d829_20': {'MAPE': 33.62188952828559},
 'HD_55e6ebb7-526e-4ee4-9441-ac4044c9d829_19': {'MAPE': 32.9024515034625},
 'HD_55e6ebb7-526e-4ee4-9441-ac4044c9d829_18': {'MAPE': 39.47979841622376},
 'HD_55e6ebb7-526e-4ee4-9441-ac4044c9d829_16': {'MAPE': 36.058521468349895},
 'HD_55e6ebb7-526e-4ee4-9441-ac4044c9d829_14': {'MAPE': 34.822428156938514},
 'HD_55e6ebb7-526e-4ee4-9441-ac4044c9d829_15': {'MAPE': 35.75074746340207},
 'HD_55e6ebb7-526e-4ee4-9441-ac4044c9d829_17': {'MAPE': 49.38677647247173},
 'HD_55e6ebb7-526e-4ee4-9441-ac4044c9d829_12': {'MAPE': 42.388894558502265},
 'HD_55e6ebb7-526e-4ee4-9441-ac4044c9d829_13': {'MAPE': 46.80077255415655},
 'HD_55e6

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [12]:
best_run_hyperdrive = hyperdrive_run.get_best_run_by_primary_metric()

# Print the best model's Run Id and metrics
best_metrics = best_run_hyperdrive.get_metrics()
print('Run Id : ', best_run_hyperdrive.id)
print('Metrics :',best_metrics)

Run Id :  HD_55e6ebb7-526e-4ee4-9441-ac4044c9d829_2
Metrics : {'MAPE': 30.704403788529106}


In [13]:
# Get the best hyperparameter values
best_hyperparameters = best_run_hyperdrive.get_details()['runDefinition']['arguments']
best_hyperparameters

['--train',
 '2126ecae-06af-47a8-9a90-f8c426bdfff1',
 '--auxi',
 '6e4656ce-f868-49a0-b0e0-4875f9eb7927',
 '--num-leaves',
 '82',
 '--min-data-in-leaf',
 '200',
 '--learning-rate',
 '0.1',
 '--feature-fraction',
 '0.7994627678278203',
 '--bagging-fraction',
 '0.5053652383845014',
 '--bagging-freq',
 '13',
 '--max-rounds',
 '1490',
 '--max-lag',
 '4',
 '--window-size',
 '21']

In [14]:
# Display the saved file of the best model
best_run_hyperdrive.get_file_names()

['azureml-logs/20_image_build_log.txt',
 'logs/azureml/dataprep/0/backgroundProcess.log',
 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/0/rslex.log.2022-07-12-18',
 'outputs/model.joblib',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/lifecycler/vm-bootstrapper.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'user_logs/std_log.txt']

In [15]:
# Download the best model
best_run_hyperdrive.download_file('outputs/model.joblib',
                                 output_file_path = './outputs/')

In [16]:
joblib.load('./outputs/model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [17]:
# Register the best model
best_hyperdrive_model = best_run_hyperdrive.register_model(model_name = 'hyperdrive-lgbm',
                                                           model_path = 'outputs/model.joblib')

In [18]:
best_hyperdrive_model

Model(workspace=Workspace.create(name='quick-starts-ws-200878', subscription_id='5a4ab2ba-6c51-4805-8155-58759ad589d8', resource_group='aml-quickstarts-200878'), name=hyperdrive-lgbm, id=hyperdrive-lgbm:1, version=1, tags={}, properties={})

In [21]:
print("Model ID : ", best_hyperdrive_model.id)

Model ID :  hyperdrive-lgbm:1


###  Create an inference config and deploy the model as a web service

In [26]:
inference_config = InferenceConfig(runtime = 'python',
                                   entry_script = 'score.py',
                                   conda_file = 'conda.yml')

In [27]:
aciconfig = AciWebservice.deploy_configuration(
    cpu_cores=2,
    memory_gb=4,
    description="LightGBM model to forecast Orange Juice data"
)

In [28]:
service = Model.deploy(
    workspace=ws,
    name="lightgbm-forecast-oj",
    models=[best_hyperdrive_model],
    inference_config=inference_config,
    deployment_config=aciconfig
)

In [29]:
service.wait_for_deployment(True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-07-12 19:11:08+00:00 Creating Container Registry if not exists.
2022-07-12 19:11:09+00:00 Registering the environment.
2022-07-12 19:11:10+00:00 Use the existing image.
2022-07-12 19:11:10+00:00 Generating deployment configuration.
2022-07-12 19:11:11+00:00 Submitting deployment to compute.
2022-07-12 19:11:16+00:00 Checking the status of deployment lightgbm-forecast-oj..
2022-07-12 19:13:17+00:00 Checking the status of inference endpoint lightgbm-forecast-oj.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [30]:
print(service.state)

Healthy


TODO: In the cell below, send a request to the web service you deployed to test it.

In [31]:
# Prepare features according to the input schema

DATA_DIR = os.path.join(os.path.abspath(os.getcwd()), "ojdata")
train_dir = os.path.join(DATA_DIR, "train")
max_lag = int(best_hyperparameters[best_hyperparameters.index("--max-lag") + 1])
lags = np.arange(2, max_lag + 1)
window_size = int(best_hyperparameters[best_hyperparameters.index("--window-size") + 1])
used_columns = [
    "store",
    "brand",
    "week",
    "week_of_month",
    "month",
    "deal",
    "feat",
    "move",
    "price",
    "price_ratio",
]
GAP = 2

In [32]:
features, end_trainng_week = create_features(
    1, train_dir, lags, window_size, used_columns
)

In [33]:
end_trainng_week

135

In [34]:
features.head(5)

store  brand  week  week_of_month  month  deal  feat    move  \
store brand                                                                   
2     1     4      2      1    44              3      7   1.0   0.0  8256.0   
            5      2      1    45              4      7   1.0   0.0  8256.0   
            6      2      1    46              5      7   0.0   0.0  6144.0   
            7      2      1    47              1      8   0.0   0.0  3840.0   
            8      2      1    48              2      8   0.0   0.0  8000.0   

                  price  price_ratio  move_lag2  move_lag3  move_lag4  \
store brand                                                             
2     1     4  0.060469     1.420889     8256.0     8256.0     8256.0   
            5  0.060469     1.420889     8256.0     8256.0     8256.0   
            6  0.060469     1.310048     8256.0     8256.0     8256.0   
            7  0.060469     1.304295     8256.0     8256.0     8256.0   
            8  0.060469     1.299913     6144.0     8256.0     8256.0   

                 move_mean  
store brand                 
2     1     4  8256.000000  
            5  8256.000000  
            6  8256.000000  
            7  8256.000000  
            8  7954.285714

In [35]:
# We will skip the week "136" to leave time for planning inventory as in real life.
# We do our forecast on the two weeks "137" and "138"

test_df = features[features.week >= end_trainng_week + GAP].reset_index(drop=True)

In [36]:
ground_truth = test_df.pop("move").values

In [48]:
# View test dataset 
print("The shape of our test dataset is : ", len(test_df))
test_df.head(5)

The shape of our test dataset is :  1826


,store,brand,week,week_of_month,month,deal,feat,price,price_ratio,move_lag2,move_lag3,move_lag4,move_mean
0,2,1,137,4,4,0.0,0.0,0.041645,1.112493,12416.0,28096.0,15168.0,15868.952381
1,2,1,138,5,4,1.0,1.0,0.037344,0.942013,12416.0,12416.0,28096.0,14829.714286
2,2,2,137,4,4,0.0,0.0,0.051979,1.388567,11424.0,4992.0,7008.0,9394.285714
3,2,2,138,5,4,1.0,1.0,0.038958,0.982741,11424.0,11424.0,4992.0,9554.285714
4,2,3,137,4,4,0.0,0.0,0.047656,1.273085,2432.0,1536.0,2816.0,2447.238095


In [40]:
# Convert data test 
data_test = json.dumps({"data": np.array(test_df).tolist()})
data_test = bytes(data_test, encoding="utf8")

In [45]:
# Send a request to the web service 
forecasts = service.run(input_data=data_test)
print("Forecasts :", forecasts)

Forecasts : [10968.21803277957, 15030.08529003189, 6742.655984891567, 11538.730539927594, 2003.3459512203563, 2476.993408396, 3425.173063310968, 7875.238246384971, 16219.988650306863, 6188.462705334735, 3568.45694391122, 1834.6585250220614, 3043.084172184779, 1986.1300612422956, 1411.690562725566, 1375.5881708792147, 1114.950429575662, 884.7281618076704, 10628.746118756273, 4564.550352361993, 4746.265575133178, 5243.303148728665, 9068.514024659296, 15508.613988253439, 7571.976158433901, 14216.235335248623, 2780.5010431371134, 3150.069726364214, 1957.09560934304, 5275.553374380627, 17655.690445033346, 6400.734410223767, 6280.907398169756, 3911.386339071836, 3877.3875769940178, 2573.4656416118783, 2073.953256536804, 2046.3479320413571, 2475.391051134332, 1976.9981901666338, 8587.221965802199, 3563.267157700495, 6691.073825137366, 9507.581600637208, 8376.263789365756, 13927.734057392256, 5962.46508477449, 11081.726481372494, 1805.9537973900783, 1838.1214256658927, 5852.861331744474, 7388.

In [46]:
# Compute MAPE metric of the test data

def compute_mape(ground_truth, forecasts) -> float:
    ground_truth, forecasts = np.array(ground_truth), np.array(forecasts)
    mape = np.mean(np.abs((ground_truth-forecasts)/ground_truth))*100
    return round(mape,2)

In [47]:
print("MAPE : ",compute_mape(ground_truth, forecasts),"%")

MAPE :  46.67 %


TODO: In the cell below, print the logs of the web service and delete the service

In [49]:
# Enable application insights
service.update(enable_app_insights=True)

In [50]:
# Print the logs of the Web service
logs = service.get_logs()
for line in logs.split('\n'):
    print(line)

2022-07-12T19:12:59,346015800+00:00 - rsyslog/run 
2022-07-12T19:12:59,353860500+00:00 - iot-server/run 
2022-07-12T19:12:59,357695400+00:00 - gunicorn/run 
2022-07-12T19:12:59,359711100+00:00 | gunicorn/run | 
2022-07-12T19:12:59,365762800+00:00 | gunicorn/run | ###############################################
2022-07-12T19:12:59,367335400+00:00 - nginx/run 
2022-07-12T19:12:59,379461700+00:00 | gunicorn/run | AzureML Container Runtime Information
2022-07-12T19:12:59,386333200+00:00 | gunicorn/run | ###############################################
2022-07-12T19:12:59,403437000+00:00 | gunicorn/run | 
2022-07-12T19:12:59,412038000+00:00 | gunicorn/run | 
2022-07-12T19:12:59,421087700+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml_1e4cf9ef955b0799eaebdf2bde3fa72f/bin:/opt/miniconda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
2022-07-12T19:12:59,428599600+00:00 | gunicorn/run | PYTHONPATH environment variable: 
2022-07-12T19:12:59,432482600+00:

### Delete

In [55]:
# Remove the web service
service.delete()

# Remove the compute cluster
compute_target.delete()

Current provisioning state of AmlCompute is "Deleting"



**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

